# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [ ]:
df= pd.read_json("world_bank_projects.json")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['country_count'] = df.groupby("countrycode").countrycode.transform('count')
df.sort_values('country_count', inplace=True,ascending=False )

In [ ]:
pd.options.display.max_columns = None
df.head(3)

## 1. Uncovering most common countries

In [ ]:
from collections import Counter
country_counter = Counter(df['countryshortname'])
country_counter.most_common(12)

## 2. Uncovering most common themes

In [ ]:
themes = [json_normalize(x) for x in df['mjtheme_namecode']]
themes_list = [pd.Series(x['name']) for x in themes]
themes = pd.concat(themes_list)
themes_counter = Counter(themes)
themes_counter.most_common(11)

## 3. Creating a dataframe with all the codes and missing values (imputed)

In [605]:
themes = [json_normalize(x) for x in df['mjtheme_namecode']]
code_list = [pd.Series(x['code']) for x in themes]
name_list = [pd.Series(x['name']) for x in themes]
code_list = pd.concat(code_list)
name_list = pd.concat(name_list)
df2 = pd.DataFrame(pd.concat([code_list,name_list], axis=1, ignore_index=True))

df2.head()

,0,1
0,9,Urban development
1,11,Environment and natural resources management
2,11,Environment and natural resources management
0,9,
0,11,Environment and natural resources management


In [606]:
df2.columns = ['code', 'name']
df2.reset_index(inplace=True)
df2['code'] = pd.to_numeric(df2['code'])
df2.isnull().sum()

index    0
code     0
name     0
dtype: int64

In [607]:
df2.loc[df2['name']=="",'name'] = None
df2.isnull().sum()

index      0
code       0
name     122
dtype: int64

In [608]:
original = df2.dropna()

original.drop_duplicates(inplace = True)
original.reset_index(inplace=True)
original = original[['code','name']]
original.head()

,code,name
0,9,Urban development
1,11,Environment and natural resources management
2,11,Environment and natural resources management
3,11,Environment and natural resources management
4,10,Rural development


In [609]:
original['code'] = pd.to_numeric(original['code'])
original.sort_values('code', inplace=True)
original.drop_duplicates(inplace = True)

In [610]:
#original.reset_index(inplace=True)
original.head()

,code,name
49,1,Economic management
27,2,Public sector governance
52,3,Rule of law
34,4,Financial and private sector development
51,5,Trade and integration


In [611]:
original = original[['code','name']]
original.head()

,code,name
49,1,Economic management
27,2,Public sector governance
52,3,Rule of law
34,4,Financial and private sector development
51,5,Trade and integration


In [612]:
df2['name'] = df2['code'].map(original.set_index('code')['name'])
df2.head(20)

,index,code,name
0,0,9,Urban development
1,1,11,Environment and natural resources management
2,2,11,Environment and natural resources management
3,0,9,Urban development
4,0,11,Environment and natural resources management
5,1,9,Urban development
6,0,11,Environment and natural resources management
7,1,11,Environment and natural resources management
8,0,11,Environment and natural resources management
9,1,11,Environment and natural resources management
